# Grouping (Basics, Pivot Tables, Crosstabs, Custom Aggregation)

### First groupby aggregation

Let's begin our usage of the `groupby` method by finding the average salary of every employee by department from the City of Houston dataset.

In [1]:
import pandas as pd
emp = pd.read_csv('../data/employee.csv')
emp.head(3)

,dept,title,hire_date,salary,sex,race
0,Police,POLICE SERGEANT,2001-12-03,87545.38,Male,White
1,Other,ASSISTANT CITY ATTORNEY II,2010-11-15,82182.00,Male,Hispanic
2,Houston Public Works,SENIOR SLUDGE PROCESSOR,2006-01-09,49275.00,Male,Black


## Exercises

### Exercise 1

<span  style="color:green; font-size:16px">Find the maximum salary for each sex.</span>

In [2]:
emp.groupby('sex').agg(max_sal = ('salary','max'))

,max_sal
sex,
Female,342784.0
Male,342784.0


### Exercise 2

<span  style="color:green; font-size:16px">Find the median salary for each department.</span>

In [3]:
(emp.groupby('dept')
    .agg(med_sal = ('salary', 'median')))

,med_sal
dept,
Fire,61921.08
Health & Human Services,50773.00
Houston Airport System,44200.00
Houston Public Works,46841.50
Library,34611.00
Other,53324.00
Parks & Recreation,31512.00
Police,68116.62
Solid Waste Management,39333.00


### Exercise 3

<span style="color:green; font-size:16px">Find the average salary for each race. Return a DataFrame with the race as a column.</span>

In [4]:
(emp.groupby('race',as_index = False)
    .agg(avg_sal = ('salary','mean')))

,race,avg_sal
0,Asian,65316.197885
1,Black,52264.180833
2,Hispanic,54811.349584
3,Native American,58153.109371
4,White,66611.692973


### Exercise 4

<span style="color:green; font-size:16px">Find the number of employees in each department.</span>

In [5]:
(emp.groupby('dept')
    .agg(num_emp = ('title', 'size')))

,num_emp
dept,
Fire,4376
Health & Human Services,1353
Houston Airport System,1216
Houston Public Works,4190
Library,563
Other,3373
Parks & Recreation,1152
Police,7573
Solid Waste Management,512


### Exercise 5

<span style="color:green; font-size:16px">Find the number of unique titles there are for each department.</span>

In [6]:
(emp.groupby('dept')
    .agg(unq_tit = ('title', 'nunique')))

,unq_tit
dept,
Fire,77
Health & Human Services,161
Houston Airport System,137
Houston Public Works,215
Library,66
Other,358
Parks & Recreation,109
Police,145
Solid Waste Management,44


### Exercise 6

<span style="color:green; font-size:16px">Find the index of the employee with the maximum salary for each department and then use those index values to select their entire rows from the original DataFrame.</span>

In [7]:
indeex = (emp.groupby('dept')
             .agg(inx_sal = ('salary','idxmax')))['inx_sal']

emp.loc[indeex]

,dept,title,hire_date,salary,sex,race
1732,Fire,"PHYSICIAN,MD",2014-09-27,342784.0,Male,White
8405,Health & Human Services,"CHIEF PHYSICIAN,MD",2017-07-31,186685.0,Female,White
3897,Houston Airport System,AVIATION DIRECTOR,2010-06-01,275000.0,Male,Hispanic
10704,Houston Public Works,PUBLIC WORKS DIRECTOR,2005-08-10,275000.0,Female,White
7564,Library,LIBRARY DIRECTOR,2005-11-07,170000.0,Female,Black
13338,Other,CITY ATTORNEY,2016-05-02,275000.0,Male,Black
11679,Parks & Recreation,PARKS & RECREATION DIRECTOR,2017-07-05,150000.0,Male,White
4413,Police,POLICE CHIEF,2016-11-30,280000.0,Male,Hispanic
20244,Solid Waste Management,SOLID WASTE DIRECTOR,2001-05-14,195000.0,Male,Black


### Use the NYC deaths dataset for the remaining exercises

Execute the cell below to read in the NYC deaths dataset and use it to answer the following exercises.

In [8]:
deaths = pd.read_csv('../data/nyc_deaths.csv')
deaths.head(3)

,year,cause,sex,race,deaths
0,2007,Accidents,F,Asian,32
1,2007,Accidents,F,Black,87
2,2007,Accidents,F,Hispanic,71


### Exercise 7

<span style="color:green; font-size:16px">What year had the most deaths?</span>

In [9]:
#Series
(deaths.groupby('year')
       .agg(per_year = ('deaths','sum'))
       .idxmax())

per_year    2008
dtype: int64

In [10]:
#DataFrame
year_deaths = deaths.groupby('year').agg(total=('deaths', 'sum'))
year_deaths.agg(['max', 'idxmax'])

,total
max,54138
idxmax,2008


### Exercise 8

<span  style="color:green; font-size:16px">Find the total number of deaths by race and sort by most to least.</span>

In [11]:
(deaths.groupby('race')
       .agg(race_death = ('deaths','sum'))
       .sort_values('race_death', ascending = False))

,race_death
race,
White,206487
Black,111116
Hispanic,74802
Asian,26355
Unknown,6238


### Exercise 9

<span  style="color:green; font-size:16px">Find the total number of deaths by cause and then select the five highest causes.</span>

In [12]:
(deaths.groupby('cause')
       .agg(death_cause = ('deaths','sum'))
       .sort_values('death_cause', ascending = False)
       .head(5))

,death_cause
cause,
Heart Disease,147551
Cancer,106367
Other,77999
Flu and Pneumonia,18678
Diabetes,13794


# Grouping and Aggregating with Multiple Columns

## Exercises
Execute the following cell to read in the City of Houston employee data and use it for the first few exercises.

In [13]:
emp = pd.read_csv('../data/employee.csv')
emp.head(3)

,dept,title,hire_date,salary,sex,race
0,Police,POLICE SERGEANT,2001-12-03,87545.38,Male,White
1,Other,ASSISTANT CITY ATTORNEY II,2010-11-15,82182.00,Male,Hispanic
2,Houston Public Works,SENIOR SLUDGE PROCESSOR,2006-01-09,49275.00,Male,Black


### Exercise 1

<span  style="color:green; font-size:16px">For each department and sex, find the number of unique position titles, the total number of employees, and the average salary. Make sure there is no multi-level index.</span>

In [14]:
(emp.groupby(['dept','sex'], as_index = False) 
    .agg(uni_pos = ('title','nunique'),
         num_emp = ('sex','size'),
         avg_sal = ('salary','mean'))
    .round({'avg_sal' : -3})
)

,dept,sex,uni_pos,num_emp,avg_sal
0,Fire,Female,51,240,62000.0
1,Fire,Male,54,4136,60000.0
2,Health & Human Services,Female,136,987,54000.0
3,Health & Human Services,Male,110,366,59000.0
4,Houston Airport System,Female,85,443,51000.0
5,Houston Airport System,Male,113,773,57000.0
6,Houston Public Works,Female,151,1195,51000.0
7,Houston Public Works,Male,180,2995,51000.0
8,Library,Female,55,404,41000.0
9,Library,Male,44,159,44000.0


### Exercise 2

<span  style="color:green; font-size:16px">For each department, race, and sex find the min and max and salaries.</span>

In [15]:
(emp.groupby(['dept','race','sex'])
    .agg(min_sal = ('salary','min'),
         max_sal = ('salary','max'))
)

min_sal    max_sal
dept                   race            sex                       
Fire                   Asian           Female  39104.0  342784.00
                                       Male    28024.0  342784.00
                       Black           Female  16411.0  342784.00
                                       Male    28024.0  342784.00
                       Hispanic        Female  28024.0   89590.02
...                                                ...        ...
Solid Waste Management Hispanic        Female  32053.0  100119.00
                                       Male    27851.0   60840.00
                       Native American Female  31325.0   31325.00
                       White           Female  36962.0  103275.00
                                       Male    31325.0   83441.00

[89 rows x 2 columns]

In [16]:
pd.set_option('display.max_columns',100)
college = pd.read_csv('../data/college.csv')
college.head(3)

,instnm,city,stabbr,hbcu,menonly,womenonly,relaffil,satvrmid,satmtmid,distanceonly,ugds,ugds_white,ugds_black,ugds_hisp,ugds_asian,ugds_aian,ugds_nhpi,ugds_2mor,ugds_nra,ugds_unkn,pptug_ef,curroper,pctpell,pctfloan,ug25abv,md_earn_wne_p10,grad_debt_mdn_supp
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,4206.0,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,11383.0,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,291.0,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370


### Exercise 3

<span  style="color:green; font-size:16px">Which city name appears the most frequently. Do this in two different ways. Do it once with and once without the `groupby` method?</span>

In [17]:
college.groupby('city').agg(size=('stabbr', 'size'))

,size
city,
Aberdeen,3
Abilene,5
Abingdon,2
Abington,1
Ada,3
...,...
Yucaipa,1
Yucca Valley,1
Yukon,1


In [18]:
college.city.value_counts().nlargest(5)

New York       87
Chicago        78
Houston        72
Los Angeles    56
Miami          51
Name: city, dtype: int64

### Exercise 4

<span style="color:green; font-size:16px">Does the city 'Houston' only appear in the state of Texas (abbreviated 'TX')?</span>

In [19]:
college.query('city == "Houston"')[['stabbr']].value_counts()

stabbr
TX        71
MO         1
dtype: int64

In [20]:
college[['city', 'stabbr']].query('city == "Houston"') \
                           .drop_duplicates(subset='stabbr')

,city,stabbr
3617,Houston,TX
5366,Houston,MO


### Exercise 5

<span style="color:green; font-size:16px">Find the maximum undergraduate population for each state?</span>

In [21]:
(college.groupby(['stabbr'])
        .agg(max_ugds = ('ugds','max'))
).head(10)

,max_ugds
stabbr,
AK,12865.0
AL,29851.0
AR,21405.0
AS,1276.0
AZ,151558.0
CA,44744.0
CO,25873.0
CT,18016.0
DC,10433.0


### Exercise 6

<span style="color:green; font-size:16px">Find the largest college from each state. From those colleges, find the difference between the largest and smallest.</span>

In [22]:
(college.groupby(['stabbr'])
        .agg(max_ugds = ('ugds','max')).agg(['max','min'])
        .diff(-1)
)

,max_ugds
max,150956.0
min,NaN


### Exercise 7

<span style="color:green; font-size:16px">Find the name and population of the largest college per state.</span>

In [23]:
c2 = college.set_index('instnm')

(c2.groupby(['stabbr'])
        .agg(max_college = ('ugds','idxmax'),
                max_ugds = ('ugds', 'max'))
).head(10)

,max_college,max_ugds
stabbr,,
AK,University of Alaska Anchorage,12865.0
AL,The University of Alabama,29851.0
AR,University of Arkansas,21405.0
AS,American Samoa Community College,1276.0
AZ,University of Phoenix-Arizona,151558.0
CA,Ashford University,44744.0
CO,University of Colorado Boulder,25873.0
CT,University of Connecticut,18016.0
DC,George Washington University,10433.0


### Exercise 8

<span  style="color:green; font-size:16px">Do distance only schools tend to have more or less student population than non-distance-only schools?</span>

In [24]:
(college.groupby('distanceonly').agg(sum_ugds = ('ugds','mean'))
)

,sum_ugds
distanceonly,
0.0,2334.648135
1.0,6245.743590


### Exercise 9

<span style="color:green; font-size:16px">Do distance only schools tend to be more or less religiously affiliated than non-distance-only schools?</span>

In [25]:
(college.groupby('distanceonly').agg(sum_ugds = ('relaffil','mean'))
)

,sum_ugds
distanceonly,
0.0,0.149635
1.0,0.050000


### Exercise 10

<span  style="color:green; font-size:16px">What state has the lowest percentage of currently operating schools of those that have religious affiliation?</span>

In [26]:
(college.query('relaffil == 1').groupby('stabbr') 
                               .agg(operating = ('curroper','mean'))
                               .nsmallest(5,'operating')
                               .round(2)
)

,operating
stabbr,
UT,0.40
AZ,0.44
NV,0.50
CA,0.59
CT,0.65


### Exercise 11

<span  style="color:green; font-size:16px">Find the top 5 historically black colleges that have the highest undergraduate white percentage (ugds_white)?</span>

In [27]:
(college.query('hbcu == 1').groupby('instnm')
                           .agg(white = ('ugds_white','mean'))
                           .nlargest(5, 'white')
)

,white
instnm,
Bluefield State College,0.8437
Gadsden State Community College,0.6921
West Virginia State University,0.5816
Shelton State Community College,0.5613
H Councill Trenholm State Community College,0.3951


# Grouping with Pivot Tables

## Exercises

Execute the following cell to read in the flights dataset and insert columns for the day and month name. Use it for the following exercises.

In [28]:
import pandas as pd
flights = pd.read_csv('../data/flights.csv', parse_dates=['date'])
flights.insert(1, 'day_of_week', flights['date'].dt.day_name())
flights.insert(2, 'month', flights['date'].dt.month_name())
flights.head(3)

,date,day_of_week,month,airline,origin,dest,dep_time,arr_time,cancelled,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2018-01-01,Monday,January,UA,LAS,IAH,100,547,0,134.0,1222.0,0,0,0,0,0
1,2018-01-01,Monday,January,WN,DEN,PHX,515,720,0,91.0,602.0,0,0,0,0,0
2,2018-01-01,Monday,January,B6,JFK,BOS,550,657,0,39.0,187.0,0,83,8,0,0


### Exercise 1

<span style="color:green; font-size:16px">What is the average carrier delay for each day of the week for each airline? Highlight the worst day of the week for each airline.</span>

In [29]:
(flights.pivot_table(index = 'airline', columns = 'day_of_week',\
                     values = 'carrier_delay', aggfunc = 'mean')
        .style.highlight_max(axis = 1)
        .format('{:.1f}'))

day_of_week,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
airline,,,,,,,
9E,3.4,4.7,1.8,3.3,7.7,3.0,4.2
AA,3.9,3.7,3.8,3.8,4.9,4.1,2.8
AS,3.0,1.5,3.6,3.7,2.9,2.6,2.1
B6,7.0,4.0,5.5,5.1,5.1,6.2,4.0
DL,3.5,2.6,3.0,3.6,3.9,3.3,3.3
EV,5.2,8.5,2.4,7.2,0.0,6.0,1.7
F9,7.9,3.6,9.5,4.1,7.8,6.6,1.8
MQ,2.1,4.2,0.0,5.4,4.2,1.5,2.3
NK,2.2,1.5,5.8,1.7,2.2,2.5,1.2


### Exercise 2

<span style="color:green; font-size:16px">Use a pivot table to find the total number of canceled flights for each origin airport and airline. Place the airlines in the columns. Use the result to find the origin airport with the most cancelled flights for each airline. Also return this maximum number of cancelled flights.</span>

In [30]:
flights.pivot_table(index = 'origin', columns = 'airline',\
                    values = 'cancelled', aggfunc = 'sum').agg(['max','idxmax'], axis = 1)

,max,idxmax
origin,,
ATL,19.0,DL
BOS,41.0,AA
CLT,33.0,AA
DCA,31.0,YX
DEN,10.0,UA
DFW,33.0,AA
DTW,8.0,DL
EWR,27.0,UA
IAH,7.0,AA


### Exercise 3

<span style="color:green; font-size:16px">Find the total distance flown for each airline for each month. Highlight the month with the most number of miles flown and use the style `format` method to put commas in the numbers so that they are easier to read.</span>

In [31]:
(flights.pivot_table(index = 'airline', columns = 'month',
                    values = 'distance', aggfunc = 'sum')
                   .style.highlight_max(axis = 1)
                   .format('{:,.0f}'))

month,April,August,December,February,January,July,June,March,May,November,October,September
airline,,,,,,,,,,,,
9E,"54,592","62,216","46,032","51,784","47,230","53,868","50,421","61,460","42,423","42,275","48,106","45,745"
AA,"1,586,655","1,649,436","1,444,276","1,371,620","1,473,883","1,669,007","1,619,325","1,528,361","1,545,453","1,409,540","1,588,285","1,482,841"
AS,"454,146","451,512","399,787","201,275","195,553","455,061","496,358","199,288","495,090","391,304","409,479","429,045"
B6,"352,234","404,458","427,097","348,189","385,517","478,230","443,151","382,666","410,877","384,038","425,712","384,008"
DL,"1,265,266","1,315,865","1,160,997","997,216","1,017,440","1,396,697","1,292,928","1,215,516","1,253,361","1,100,681","1,214,950","1,173,359"
EV,"6,847","1,194","3,933","11,854","10,186",927,"5,926","4,511","3,569","1,592","2,587",995
F9,"117,439","97,777","97,846","97,879","118,067","84,417","116,116","80,444","78,807","110,423","105,833","89,938"
MQ,"13,060","15,787","14,057","17,539","15,170","20,057","15,310","13,349","13,656","15,559","16,884","14,767"
NK,"250,683","270,894","232,613","219,678","249,461","273,963","318,648","228,829","261,421","266,838","253,692","235,754"


### Exercise 4

<span style="color:green; font-size:16px">Create a pivot table that shows the number of flights flown for every day of the week for every month.</span>

In [32]:
(flights.pivot_table(index = 'day_of_week', columns = 'month',
                    aggfunc = 'size'))

month,April,August,December,February,January,July,June,March,May,November,October,September
day_of_week,,,,,,,,,,,,
Friday,786,1006,707,748,673,808,1005,974,766,887,773,739
Monday,943,785,840,725,862,936,792,734,798,710,961,742
Saturday,644,592,750,544,536,677,817,705,597,624,554,742
Sunday,898,776,897,639,696,932,809,687,726,737,711,902
Thursday,818,982,759,753,739,765,842,884,1058,871,790,761
Tuesday,755,757,634,716,821,1012,779,676,913,707,939,719
Wednesday,754,963,695,719,838,780,822,751,894,709,894,762


### Exercise 5

<span style="color:green; font-size:16px">In exercise 4, the months and days of week are ordered alphabetically. It would be better if these values were ordered chronologically. Can you return a result that has both groups in the correct order. Use Monday as the first day of the week.</span>

In [33]:
month_dtype = pd.CategoricalDtype(['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August', 'September', 'October', 'November', 'December'], ordered = True)

day_dtype = pd.CategoricalDtype(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday',
       'Sunday'], ordered = True)

flights = flights.astype({'day_of_week' : day_dtype, 'month' : month_dtype})

In [34]:
flights.pivot_table(index = 'day_of_week', columns = 'month',\
                   values = 'dest', aggfunc = 'size', sort = False)

month,January,February,March,April,May,June,July,August,September,October,November,December
day_of_week,,,,,,,,,,,,
Monday,862,725,734,943,798,792,936,785,742,961,710,840
Tuesday,821,716,676,755,913,779,1012,757,719,939,707,634
Wednesday,838,719,751,754,894,822,780,963,762,894,709,695
Thursday,739,753,884,818,1058,842,765,982,761,790,871,759
Friday,673,748,974,786,766,1005,808,1006,739,773,887,707
Saturday,536,544,705,644,597,817,677,592,742,554,624,750
Sunday,696,639,687,898,726,809,932,776,902,711,737,897


### Exercise 6

<span style="color:green; font-size:16px">Create a new column in the flights dataset called `'dep_time_hour'` and set it equal to the hour (this will be an integer 0 through 23) of the flight. Find the average carrier delay for every month and dep_time_hour. Place the month in the columns.</span>

In [35]:
flights['dep_time_hour'] =flights['dep_time'] // 100 
(flights.pivot_table(index = 'dep_time_hour', columns = 'day_of_week',
                    values = 'carrier_delay', aggfunc = 'mean')
        .round(1))

day_of_week,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
dep_time_hour,,,,,,,
0,0.2,3.6,1.2,2.4,6.5,1.7,2.4
1,0.0,0.0,1.1,0.7,0.0,1.7,2.3
2,NaN,NaN,0.0,NaN,NaN,0.0,0.0
4,NaN,NaN,NaN,NaN,0.0,NaN,NaN
5,5.9,1.5,1.8,2.4,4.4,3.0,1.8
6,2.5,2.8,2.6,2.8,5.2,4.0,3.5
7,2.0,3.5,1.8,3.7,3.8,4.1,1.8
8,3.6,3.1,7.2,4.3,2.3,4.0,3.7
9,3.1,2.8,2.5,3.8,3.3,3.4,3.0


### Exercise 7

<span style="color:green; font-size:16px">Use both `groupby` and `pivot_table` to compute the average and median distance flown by day of the week.</span>

In [36]:
flights.groupby('day_of_week').agg(median_dist=('distance', 'median'),
                                   mean_dist=('distance', 'mean'))

,median_dist,mean_dist
day_of_week,,
Monday,912.0,1071.351038
Tuesday,888.0,1051.831141
Wednesday,868.0,1052.546290
Thursday,907.0,1066.285073
Friday,868.0,1050.703404
Saturday,937.0,1107.159728
Sunday,925.0,1092.702657


In [37]:
(flights.pivot_table(index = 'day_of_week', \
                   values = 'distance', aggfunc = ['median', 'mean']))

,median,mean
,distance,distance
day_of_week,,
Monday,912.0,1071.351038
Tuesday,888.0,1051.831141
Wednesday,868.0,1052.546290
Thursday,907.0,1066.285073
Friday,868.0,1050.703404
Saturday,937.0,1107.159728
Sunday,925.0,1092.702657


### Exercise 8

<span style="color:green; font-size:16px">Use a dictionary in the `groupby` `agg` method to calculate the mean, median, min, and max of the air time for every airline.</span>

In [38]:
flights.groupby('airline').agg({'arr_time' : ['mean','median','min','max']})

arr_time                   
                mean  median  min   max
airline                                
9E       1501.621022  1449.0  657  2356
AA       1488.175815  1527.0    1  2359
AS       1478.302404  1536.0    1  2359
B6       1456.797170  1503.0    1  2400
DL       1520.030754  1542.0    2  2359
EV       1494.584795  1549.0  712  2200
F9       1453.113935  1543.0    2  2359
MQ       1541.801609  1600.0  620  2325
NK       1485.779305  1521.5    1  2400
OH       1572.451362  1639.0  747  2340
OO       1483.955875  1527.0    4  2359
UA       1487.313163  1525.0    1  2359
VX       1460.004662  1546.0    3  2359
WN       1488.442590  1530.0    5  2355
YV       1484.489712  1523.0    1  2353
YX       1524.359688  1529.0   10  2359

### Exercise 9

<span style="color:green; font-size:16px">Without using the `agg` method calculate the number of unique destinations for each airline.</span>

In [39]:
flights.groupby('airline')[['dest']].nunique()

,dest
airline,
9E,13
AA,20
AS,18
B6,19
DL,20
EV,8
F9,17
MQ,12
NK,16


### Exercise 10

<span style="color:green; font-size:16px">Calculate the mean of every numeric column for each airline and origin without using the `agg` method.</span>

In [40]:
flights.groupby(['airline','origin']).mean(numeric_only = True)

dep_time     arr_time  cancelled    air_time     distance  \
airline origin                                                                 
9E      ATL      729.714286   840.142857   0.000000  105.142857   689.000000   
        BOS     1320.029412  1453.970588   0.049020   47.103093   191.558824   
        CLT     1254.613636  1461.238636   0.068182   82.926829   554.397727   
        DCA     1168.218182  1309.563636   0.018182   44.629630   216.490909   
        DFW     1346.068182  1682.136364   0.011364  137.080460  1054.022727   
...                     ...          ...        ...         ...          ...   
YX      JFK     1519.901961  1677.705882   0.078431   57.191489   278.725490   
        LGA     1272.184569  1456.931921   0.046899   91.452229   546.311649   
        MSP     1167.055556  1494.888889   0.037037  114.737179   843.049383   
        ORD     1374.639535  1515.186047   0.027132   86.163347   589.135659   
        PHL     1515.984375  1690.390625   0.000000   71.359375   391.703125   

                carrier_delay  weather_delay  nas_delay  security_delay  \
airline origin                                                            
9E      ATL          0.000000       0.000000   3.857143             0.0   
        BOS          2.284314       0.029412   8.607843             0.0   
        CLT          5.261364       0.000000   4.022727             0.0   
        DCA         10.945455       0.000000   1.309091             0.0   
        DFW          2.840909       0.738636   3.772727             0.0   
...                       ...            ...        ...             ...   
YX      JFK          0.000000       0.000000   3.078431             0.0   
        LGA          3.028744       0.422088   3.836611             0.0   
        MSP          1.265432       0.037037   7.962963             0.0   
        ORD          2.093023       0.352713   7.015504             0.0   
        PHL          3.984375       0.031250   2.140625             0.0   

                late_aircraft_delay  dep_time_hour  
airline origin                                      
9E      ATL                0.000000       7.000000  
        BOS                4.176471      13.049020  
        CLT                4.295455      12.352273  
        DCA                2.163636      11.527273  
        DFW               11.875000      13.193182  
...                             ...            ...  
YX      JFK                2.980392      14.960784  
        LGA                4.751891      12.541604  
        MSP                5.067901      11.388889  
        ORD                7.569767      13.550388  
        PHL                4.000000      14.875000  

[240 rows x 11 columns]

# Custom Aggregation

## Exercises

### Exercise 1

<span style="color:green; font-size:16px">What are the three airlines with the least number of flights?</span>

In [41]:
flights.groupby('airline')['origin'].size().nsmallest(3)

airline
EV    171
OH    257
MQ    373
Name: origin, dtype: int64

### Exercise 2

<span style="color:green; font-size:16px">For each airline, find the 75th percentile of flight distance. Use a custom aggregation function.</span>

In [42]:
def seventh_per(s):
    return s.quantile(.75)

In [43]:
flights.groupby('airline').agg(percentile_75 = ('distance', seventh_per))

,percentile_75
airline,
9E,852.0
AA,1558.0
AS,2402.0
B6,2381.0
DL,1587.0
EV,514.5
F9,1476.0
MQ,612.0
NK,1379.0


### Exercise 3

<span style="color:green; font-size:16px">For each airline, find out what percentage of its flights leave on a Tuesday. Use a custom aggregation function.</span>

In [44]:
def tuesday_flights(s):
    return (s.dt.day_name() == 'Tuesday').mean()

In [45]:
flights.groupby('airline').agg(pct_tuesday = ('date',tuesday_flights))

,pct_tuesday
airline,
9E,0.144648
AA,0.146195
AS,0.138101
B6,0.134958
DL,0.143544
EV,0.157895
F9,0.128834
MQ,0.160858
NK,0.128437


### Exercise 4

<span style="color:green; font-size:16px">Optimize exercise 2 without using a custom aggregation. What is the performance difference?</span>

In [46]:
flights['is_tues'] = flights['date'].dt.day_name() == 'Tuesday'
flights.groupby('airline').agg(pct_tuesday = ('is_tues', 'mean'))

,pct_tuesday
airline,
9E,0.144648
AA,0.146195
AS,0.138101
B6,0.134958
DL,0.143544
EV,0.157895
F9,0.128834
MQ,0.160858
NK,0.128437


### Exercise 5

<span style="color:green; font-size:16px">The range of salaries per department was calculated using the `min_max` custom function from the beginning of this chapter. Use this same function to calculate the range of distance for each airline. Then calculate this range again without a custom function.</span>

In [47]:
def min_max(s):
    return s.max() - s.min()

In [48]:
flights.groupby('airline').agg(in_ax = ('distance', min_max))

,in_ax
airline,
9E,1297.0
AA,2515.0
AS,2468.0
B6,2520.0
DL,2610.0
EV,876.0
F9,2042.0
MQ,831.0
NK,2166.0


### Exercise 6

<span style="color:green; font-size:16px">Which origin airport has the highest percentage of its flights cancelled?</span>

In [49]:
(flights.groupby('origin').agg(pct_cancelled = ('cancelled','mean'))
                          .nlargest(1, 'pct_cancelled')
                          .round(3) * 100)

,pct_cancelled
origin,
BOS,3.4


# Transform and Filter with Groupby

## Exercises

Execute the cell below to reread the college dataset and use it for the exercises below.

In [50]:
cols = ['instnm', 'stabbr', 'relaffil', 'satvrmid', 'satmtmid', 'ugds']
college = pd.read_csv('../data/college.csv', usecols=cols, index_col='instnm')
college.head(3)

,stabbr,relaffil,satvrmid,satmtmid,ugds
instnm,,,,,
Alabama A & M University,AL,0,424.0,420.0,4206.0
University of Alabama at Birmingham,AL,0,570.0,565.0,11383.0
Amridge University,AL,1,NaN,NaN,291.0


### Exercise 1

<span style="color:green; font-size:16px">Filter the college DataFrame for states that have more than 500,000 total undergraduate students. Can you verify your results?</span>

In [51]:
def more_500(s):
    return s['ugds'].sum() > 500000

In [52]:
col = college.groupby('stabbr').filter(more_500)
col.head()

,stabbr,relaffil,satvrmid,satmtmid,ugds
instnm,,,,,
Prince Institute-Southeast,IL,0,NaN,NaN,84.0
Everest College-Phoenix,AZ,1,NaN,NaN,4102.0
Collins College,AZ,0,NaN,NaN,83.0
Empire Beauty School-Paradise Valley,AZ,1,NaN,NaN,25.0
Empire Beauty School-Tucson,AZ,0,NaN,NaN,126.0


In [53]:
col.groupby('stabbr')['ugds'].sum()

stabbr
AZ     520439.0
CA    2304492.0
FL     959753.0
IL     599816.0
NY     993623.0
OH     537638.0
PA     604942.0
TX    1277374.0
Name: ugds, dtype: float64

### Exercise 2

<span style="color:green; font-size:16px">Filter the college DataFrame for states that have a an average undergraduate student population greater than 2,500 and have more than 30 religiously affiliated schools. Can you verify your results?</span>

In [54]:
def ugds_reg(s):
    return s['ugds'].mean() > 2500 and s['relaffil'].sum() > 30

In [55]:
college.groupby('stabbr').filter(ugds_reg)

,stabbr,relaffil,satvrmid,satmtmid,ugds
instnm,,,,,
Academy of Art University,CA,0,NaN,NaN,9885.0
ITT Technical Institute-Rancho Cordova,CA,0,NaN,NaN,500.0
Academy of Chinese Culture and Health Sciences,CA,0,NaN,NaN,NaN
The Academy of Radio and TV Broadcasting,CA,0,NaN,NaN,14.0
Avalon School of Cosmetology-Alameda,CA,0,NaN,NaN,253.0
...,...,...,...,...,...
WestMed College - Merced,CA,1,NaN,NaN,NaN
Vantage College,TX,1,NaN,NaN,NaN
SAE Institute of Technology San Francisco,CA,1,NaN,NaN,NaN


### Exercise 3

<span style="color:green; font-size:16px">The maximum SAT score for each test is 800. Create a new column in the college dataset that shows each school's percentage of maximum for each SAT score.</span>

In [56]:
def sat_800(s):
    return (s / 800).round(3) * 100

In [57]:
college[['satvrmid_pct', 'satmtmid_pct']] = (college[['satvrmid','satmtmid']] / 800).round(2) * 100
college

,stabbr,relaffil,satvrmid,satmtmid,ugds,satvrmid_pct,satmtmid_pct
instnm,,,,,,,
Alabama A & M University,AL,0,424.0,420.0,4206.0,53.0,52.0
University of Alabama at Birmingham,AL,0,570.0,565.0,11383.0,71.0,71.0
Amridge University,AL,1,NaN,NaN,291.0,NaN,NaN
University of Alabama in Huntsville,AL,0,595.0,590.0,5451.0,74.0,74.0
Alabama State University,AL,0,425.0,430.0,4811.0,53.0,54.0
...,...,...,...,...,...,...,...
SAE Institute of Technology San Francisco,CA,1,NaN,NaN,NaN,NaN,NaN
Rasmussen College - Overland Park,KS,1,NaN,NaN,NaN,NaN,NaN
National Personal Training Institute of Cleveland,OH,1,NaN,NaN,NaN,NaN,NaN


### Use the City of Houston dataset

Execute the following cell to read in the City of Houston employee dataset and then use it for the following exercises.

In [58]:
emp = pd.read_csv('../data/employee.csv')
emp.head(3)

,dept,title,hire_date,salary,sex,race
0,Police,POLICE SERGEANT,2001-12-03,87545.38,Male,White
1,Other,ASSISTANT CITY ATTORNEY II,2010-11-15,82182.00,Male,Hispanic
2,Houston Public Works,SENIOR SLUDGE PROCESSOR,2006-01-09,49275.00,Male,Black


### Exercise 4

<span style="color:green; font-size:16px">Filter it so that only position titles with an average salary of 100,000 remain. Can you verify your results?</span>

In [59]:
def avg_100k(s):
    return s.mean() > 100000

In [60]:
emp.groupby('title')[['salary']].filter(avg_100k)

,salary
16,107744.00
17,115821.42
19,95783.00
39,112270.00
48,115835.98
...,...
24159,342784.00
24219,92104.00
24238,162915.00
24267,98703.00


In [61]:
emp.groupby('title').filter(lambda df: df['salary'].mean() > 100000)

,dept,title,hire_date,salary,sex,race
16,Other,ASSOCIATE JUDGE OF MUNICIPAL COURTS,2005-11-09,107744.00,Male,Hispanic
17,Police,POLICE COMMANDER,1983-02-07,115821.42,Male,White
19,Other,ASSISTANT DIRECTOR (EXECUTIVE LEVEL),2002-05-28,95783.00,Female,Hispanic
39,Houston Airport System,DEPUTY ASSISTANT DIRECTOR (EXECUTIVE LEV,2017-08-15,112270.00,Male,Black
48,Fire,ASSISTANT FIRE CHIEF,1994-11-07,115835.98,Male,Hispanic
...,...,...,...,...,...,...
24159,Fire,"PHYSICIAN,MD",2017-01-09,342784.00,Male,Asian
24219,Other,ERP BUSINESS SYSTEMS CONSULTANT,2001-07-09,92104.00,Female,White
24238,Other,DEPUTY CIO - IT INFRASTRUCTURE (EXE LVL),2006-12-04,162915.00,Male,White
24267,Houston Public Works,SUPERVISING ENGINEER,2011-03-21,98703.00,Female,Asian


### Exercise 5

<span style="color:green; font-size:16px">Filter the employee dataset so that only position titles with at least 5 employees and an average salary of 80,000 remain. Can you verify the results?</span>

In [62]:
def avg_80k(s):
    return s.mean() > 80000 and len(s) >= 5

In [63]:
emp.groupby('title')['salary'].filter(avg_80k)

0         87545.38
1         82182.00
16       107744.00
17       115821.42
19        95783.00
           ...    
24271     85372.00
24276     89623.00
24288     89590.02
24292     95004.00
24307          NaN
Name: salary, Length: 3172, dtype: float64

In [64]:
emp.groupby('title').filter(lambda df: len(df) >= 5 and df['salary'].mean() > 80000)

,dept,title,hire_date,salary,sex,race
0,Police,POLICE SERGEANT,2001-12-03,87545.38,Male,White
1,Other,ASSISTANT CITY ATTORNEY II,2010-11-15,82182.00,Male,Hispanic
16,Other,ASSOCIATE JUDGE OF MUNICIPAL COURTS,2005-11-09,107744.00,Male,Hispanic
17,Police,POLICE COMMANDER,1983-02-07,115821.42,Male,White
19,Other,ASSISTANT DIRECTOR (EXECUTIVE LEVEL),2002-05-28,95783.00,Female,Hispanic
...,...,...,...,...,...,...
24271,Other,DIVISION MANAGER,1989-10-30,85372.00,Male,Black
24276,Other,DIVISION MANAGER,1993-09-21,89623.00,Male,Black
24288,Fire,DISTRICT CHIEF,1982-06-28,89590.02,Male,White
24292,Houston Airport System,SENIOR STAFF ANALYST (EXECUTIVE LEVEL),2018-11-19,95004.00,Male,Black


### Exercise 6

<span style="color:green; font-size:16px">Add a column to the DataFrame that contains the median salary based on department, sex, and race.</span>

In [65]:
emp['med_sal_dept_sex_race'] = emp.groupby(['dept',  'sex', 'race'])['salary'].transform('median')
emp.head(10)

,dept,title,hire_date,salary,sex,race,med_sal_dept_sex_race
0,Police,POLICE SERGEANT,2001-12-03,87545.38,Male,White,73479.00
1,Other,ASSISTANT CITY ATTORNEY II,2010-11-15,82182.00,Male,Hispanic,47445.00
2,Houston Public Works,SENIOR SLUDGE PROCESSOR,2006-01-09,49275.00,Male,Black,38813.00
3,Police,SENIOR POLICE OFFICER,1997-05-27,75942.10,Male,Hispanic,68116.62
4,Police,SENIOR POLICE OFFICER,2006-01-23,69355.26,Male,White,73479.00
5,Other,SENIOR ACCOUNT CLERK,2017-10-09,44616.00,Female,Black,52915.00
6,Houston Public Works,ADMINISTRATIVE ASSISTANT,2005-08-29,39998.00,Female,Black,39707.00
7,Police,POLICE OFFICER,2008-03-31,68116.62,Male,Asian,65755.00
8,Fire,FIRE FIGHTER,2015-08-01,48189.70,Male,White,62540.14
9,Police,SENIOR POLICE OFFICER,1993-08-30,75942.10,Male,Black,68116.62


### Exercise 7

<span  style="color:green; font-size:16px">Add a new column, `pct_max_dept_sex`, to the employee DataFrame that holds the employees percentage of the maximum salary for each department and sex. For instance, if a male HPD employee makes 80,000 and the maximum male HPD salary is 120,000 then the value for this employee would be 80,000/120,000 or .666. Verify this value for the first employee.</span>

In [66]:
def max_percent(s):
    return s / s.max()

In [67]:
emp['pct_max_dept_sex']  = emp.groupby(['dept', 'sex'])['salary'].transform(max_percent)
emp['pct_max_dept_sex']

0        0.312662
1        0.298844
2        0.227809
3        0.271222
4        0.247697
           ...   
24303    0.271222
24304    0.371585
24305    0.162612
24306    0.359954
24307         NaN
Name: pct_max_dept_sex, Length: 24308, dtype: float64